In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.tabular import *

In [4]:
from pathlib import Path

In [5]:
path = Path('/root/.fastai/dataautovista_tabular')

In [6]:
path

PosixPath('/root/.fastai/dataautovista_tabular')

In [7]:
df = pd.read_csv(path/'autoscout.it-2019-07-31-0200.csv.extract.csv', sep='|')

In [8]:
df = df[df['VehicleCategory'] == 'Car']

In [9]:
df['FiscalHorsePower'] = df['FiscalHorsePower'].replace({'[^\d]+' : ''}, regex=True)
df['FiscalHorsePower'] = pd.to_numeric(df['FiscalHorsePower'])

In [10]:
df.head()

,CountryCode,LanguageCode,Town,RegistrationDate,YearOfManufacture,Make,Model,Type,Trim,FuelType,...,MILEAGE,MileageUnits,PRICE,NumberOfOwners,FourWheelDrive,OPTIONS,VehicleCategory,CO2Emissions,FiscalHorsePower,Seats
0,IT,ITIT,Milano - Mi,1.498864e+09,2017,Porsche,911,3.0 Carrera 4S Cabriolet,NaN,Benzina,...,10360.0,KM,118900.0,0,1.0,"Climatizzatore,Volante in pelle,Volante riscal...",Car,208.0,420.0,4.0
1,IT,ITIT,Busto Arsizio,1.548979e+09,2019,Fiat,500,1.2 S,NaN,Benzina,...,2200.0,KM,10500.0,0,NaN,"Vetri oscurati,Volante in pelle,Start/Stop Aut...",Car,123.0,69.0,4.0
2,IT,ITIT,"Vigorovea, Sant'Angelo di Piove di Sacco- Pd",1.296518e+09,2011,Dacia,Duster,1.5 dCi 110CV 4x2 Lauréate,NaN,Diesel,...,149000.0,KM,6290.0,0,0.0,"Climatizzatore,Alzacristalli elettrici,Autorad...",Car,135.0,110.0,5.0
3,IT,ITIT,Genova - Ge,1.275347e+09,2010,Volkswagen,Tiguan,4Motion Sport&Style,NaN,Diesel,...,121000.0,KM,9800.0,0,1.0,"Hill Holder,Sensore di luminosità,Climatizzato...",Car,0.0,140.0,5.0
4,IT,ITIT,Bologna - Bo,0.000000e+00,0,Opel,Mokka,X 1.6 CDTI Ecotec 136CV 4x2 Start&Stop Vision,NaN,Diesel,...,0.0,KM,22500.0,0,0.0,"Climatizzatore,Cruise Control,Alzacristalli el...",Car,131.0,136.0,5.0


In [11]:
df.shape

(418425, 23)

In [12]:
df['FiscalHorsePower'].unique()

array([ 420.,   69.,  110.,  140., ...,  339.,  598., 2484., 1043.])

In [13]:
dep_var = 'PRICE'
cat_names = ['Make', 'Model', 'Trim', 'FuelType', 'BodyType', 'DOORS', 'Type', 'FourWheelDrive', 'Seats']
cont_names = ['RegistrationDate', 'YearOfManufacture', 'EngineSize', 'MILEAGE', 'FiscalHorsePower']
procs = [FillMissing, Categorify, Normalize]

In [14]:
df.shape[0] * 0.1

41842.5

In [15]:
range_start = 10000
range_end = int(10000 + df.shape[0] * 0.1)

In [16]:
test = TabularList.from_df(df.iloc[range_start:range_end].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [17]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(range_start,range_end)))
                           .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
                           .add_test(test)
                           .databunch())

In [18]:
data.show_batch(rows=10)

Make,Model,Trim,FuelType,BodyType,DOORS,Type,FourWheelDrive,Seats,EngineSize_na,FiscalHorsePower_na,RegistrationDate,YearOfManufacture,EngineSize,MILEAGE,FiscalHorsePower,target
Jeep,Renegade,#na#,Benzina,SUV/Fuoristrada/Pick-up,5.0,1.0 t3 Longitude fwd,0.0,5.0,False,False,0.7613,0.2560,-1.1727,-0.0187,-0.1964,9.846917
Dodge,RAM,#na#,Benzina,SUV/Fuoristrada/Pick-up,5.0,LongBed Laramie 6.4 box DETRAZIONE 130%,1.0,5.0,False,False,-3.1836,-4.1654,6.0022,-0.0187,3.8205,10.757903
Audi,A6,#na#,Elettrica/Diesel,Berlina,4.0,45 3.0 TDI quattro tiptronic Business Sport,1.0,5.0,False,False,0.7082,0.2538,1.8607,-0.0187,1.3904,11.080603
Ford,EcoSport,#na#,Diesel,SUV/Fuoristrada/Pick-up,5.0,1.5 TDCi 100 CV Start&Stop ST-Line Plus,0.0,5.0,False,False,0.6881,0.2538,-0.4036,-0.0148,-0.4966,9.797571
Fiat,Punto,#na#,Benzina,City car,3.0,Classic 1.2 3 porte OK neopatentati,0.0,5.0,False,False,-0.1301,0.2319,-0.7981,-0.0074,-1.0541,8.242756
Kia,Sorento,#na#,Diesel,SUV/Fuoristrada/Pick-up,5.0,2.5 16V CRDI VGT 4WD KM 110.000 TAGLIANDATI CERT.,1.0,5.0,False,False,-0.1967,0.2297,1.1362,0.0035,0.5184,8.824677
BMW,X3,#na#,Benzina,SUV/Fuoristrada/Pick-up,5.0,xDrive 20i Msport,1.0,5.0,False,False,0.7412,0.2560,0.3671,-0.0187,0.7185,10.857074
Volkswagen,Golf,#na#,Diesel,Berlina,5.0,#na#,0.0,#na#,False,False,-0.5825,0.2188,0.2161,0.0520,-0.2535,6.856462
Hyundai,Tucson,#na#,Diesel,SUV/Fuoristrada/Pick-up,5.0,1.7 CRDi XPossible,#na#,5.0,False,False,0.5688,0.2516,-0.1153,-0.0135,-0.2535,10.064755
BMW,420,#na#,Diesel,Coupé,5.0,Serie 4 Gran Coupé Msport / PELLE/VIRTUAL/PROF,0.0,5.0,False,False,0.6682,0.2538,0.3625,-0.0138,0.8043,10.616437


In [19]:
max_log_y = np.log(np.max(df['PRICE'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [20]:
learn = tabular_learner(data, layers=[1000, 250], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=root_mean_squared_error)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(3, 1e-3)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('first')

In [21]:
learn.load('first')

Learner(data=TabularDataBunch;

Train: LabelList (376583 items)
x: TabularList
Make Porsche; Model 911; Trim #na#; FuelType Benzina; BodyType Cabrio; DOORS 2.0; Type 3.0 Carrera 4S Cabriolet; FourWheelDrive 1.0; Seats 4.0; EngineSize_na False; FiscalHorsePower_na False; RegistrationDate 0.6083; YearOfManufacture 0.2516; EngineSize 1.8822; MILEAGE -0.0166; FiscalHorsePower 4.0921; ,Make Fiat; Model 500; Trim #na#; FuelType Benzina; BodyType Berlina; DOORS 3.0; Type 1.2 S; FourWheelDrive #na#; Seats 4.0; EngineSize_na False; FiscalHorsePower_na False; RegistrationDate 0.7351; YearOfManufacture 0.2560; EngineSize -0.7981; MILEAGE -0.0183; FiscalHorsePower -0.9254; ,Make Dacia; Model Duster; Trim #na#; FuelType Diesel; BodyType SUV/Fuoristrada/Pick-up; DOORS 5.0; Type 1.5 dCi 110CV 4x2 Lauréate; FourWheelDrive 0.0; Seats 5.0; EngineSize_na False; FiscalHorsePower_na False; RegistrationDate 0.0964; YearOfManufacture 0.2385; EngineSize -0.4606; MILEAGE 0.0114; FiscalHorsePower -0.3393; ,Make

### Predictions

In [22]:
row = df.iloc[0]

In [24]:
row

CountryCode                                                         IT
LanguageCode                                                      ITIT
Town                                                       Milano - Mi
RegistrationDate                                           1.49886e+09
YearOfManufacture                                                 2017
Make                                                           Porsche
Model                                                              911
Type                                          3.0 Carrera 4S Cabriolet
Trim                                                               NaN
FuelType                                                       Benzina
BodyType                                                        Cabrio
DOORS                                                                2
EngineSize                                                        2981
MILEAGE                                                          10360
Mileag

In [25]:
np.exp(learn.predict(row)[1].data[0].item())

113068.9035641907

In [26]:
for i in range(20):
    r = df.iloc[i]
    price = r.PRICE
    prediction = np.exp(learn.predict(r)[1].data[0].item())
    print(f'{price: <10} {prediction: <30}')

118900.0   113068.9035641907             
10500.0    12349.71946111136             
6290.0     6700.685634608472             
9800.0     9834.577611423836             
22500.0    20921.828462310234            
5400.0     5442.768641609127             
26800.0    27291.814943789595            
5850.0     6675.147775693727             
4712.0     4311.340905051571             
6500.0     5465.254048149292             
17800.0    19511.9665587519              
1800.0     2494.2741027059906            
27000.0    25137.81878157424             
12000.0    9106.937139977023             
5500.0     5620.357230794441             
18800.0    18337.751690808043            
18990.0    18462.145859718978            
39000.0    33245.27144503543             
12880.0    13068.606934111422            
19850.0    19188.240904137427            
